In [1]:
from sqlalchemy import create_engine, text, bindparam, Integer
from sqlalchemy.orm import sessionmaker
import pandas as pd

In [149]:
class ArticuloProveedor:
    def __init__(self):
        self.server_solmicro = r'srvsql'
        self.server_industry = r'SERVIDOR'
        self.database_solmicro = 'SolmicroERP6_PruebasSub'
        self.database_industry = 'IPFavram'
        self.username_solmicro = 'sa'
        self.password_solmicro = 'Altai2021'
        self.password_industry = '71zl6p9h'
        self.connection_string_solmicro = create_engine(
            f'mssql+pyodbc://{self.username_solmicro}:{self.password_solmicro}@{self.server_solmicro}/{self.database_solmicro}?driver=SQL+Server')
        self.connection_string_industry = create_engine(
            f'mssql+pyodbc://{self.username_solmicro}:{self.password_industry}@{self.server_industry}/{self.database_industry}?driver=SQL+Server')
        self.connection = None
        self.connection_industry = None
        self.connection_solmicro = f'DRIVER={{SQL Server}};SERVER={self.server_solmicro};DATABASE={self.database_solmicro};UID={self.username_solmicro};PWD={self.password_solmicro}'
        self.ruta_archivo = r'ArtiConProveedorIndustry.xlsx' 
        self.hoja_excel = 'Sheet1'  

    def CargarExcel(self,nameColumn):
        try:
            datos_excel = pd.read_excel(self.ruta_archivo, sheet_name=self.hoja_excel)
            columna_especifica = datos_excel[nameColumn] 
            art_ids = list(set(round(item) for item in columna_especifica if pd.notnull(item)))
            return art_ids
        except Exception as e:
            print("Error Excel: ",e)
    
    def ExcelArtProveedor(self,listProvedore):
        try:
            datos_excel = pd.read_excel(self.ruta_archivo, sheet_name=self.hoja_excel)
            ProveedorHabitual = datos_excel["ProveedorHabitual"]
            CodigoArticulo = datos_excel["CodigoArticulo"]
            filtered_data = datos_excel[ProveedorHabitual.isin(listProvedore)]
            art_codigos_proveedores = list(zip(filtered_data["CodigoArticulo"], round(filtered_data["ProveedorHabitual"])))
            return art_codigos_proveedores
        except Exception as e:
            print("Error Excel: ",e)

    
    def Open_Conn_Solmicro(self):
        try:
            self.connection = self.connection_string_solmicro.connect()
            return self.connection
        except Exception as e:
            print("Error opening connection: ", e)

    def Open_Conn_Industry(self):
        try:
            self.connection = self.connection_string_industry.connect()
            return self.connection
        except Exception as e:
            print("Error opening connection: ", e)

    def getArtCompra(self):
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                query = text(f"select IDArticulo from tbMaestroArticulo where IDTipo = '04' AND IDFamilia>N'4000' AND IDFamilia <N'4200' AND IDFamilia <>N'4020'")
                result = conn.execute(query).fetchall()
                conn.close()
                list_ids = [row[0] for row in result] if result else []
            return list_ids
        except Exception as e:
            print("Error en la consulta:", e)
            if conn:
                conn.close()

    def getArtCompraConProveedor(self):
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                query = text(f"SELECT tbMaestroArticulo.IDArticulo FROM tbMaestroArticulo INNER JOIN tbArticuloProveedor ON tbMaestroArticulo.IDArticulo = tbArticuloProveedor.IDArticulo WHERE (tbMaestroArticulo.IDTipo = N'04') AND (tbMaestroArticulo.IDFamilia > N'4000') AND (tbMaestroArticulo.IDFamilia < N'4200') AND (tbMaestroArticulo.IDFamilia <>N'4020')")
                result = conn.execute(query).fetchall()
                conn.close()
                list_ids = [row[0] for row in result] if result else []
            return list_ids
        except Exception as e:
            print("Error en la consulta:", e)
            if conn:
                conn.close()

    def getProArtConProveeDesdeIndustry(self,list_ids):
        resultados = []
        try:
            conn = self.Open_Conn_Industry()
            if conn:
                for id in list_ids:
                    query = text(f"SELECT CodigoArticulo, ProveedorHabitual, PrecioCosteStandard, PlazoAprovisionam FROM MArticulo WHERE (CodigoArticulo = N'{id}')")
                    result = conn.execute(query).fetchall()
                    if result:
                        resultados.extend(result)
                conn.close()
            return resultados
        except Exception as e:
            print("Error en la consulta:", e)
            if conn:
                conn.close()

    def getCheckProveedor(self, listaProveedor):
        resultados = []
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                for proveedor in listaProveedor:
                    query = text(f"SELECT top(1) IDProveedor FROM tbMaestroProveedor WHERE (IDProveedor = N'{proveedor}')")
                    result = conn.execute(query).fetchall()
                    if not result:
                        resultados.append(proveedor)
                conn.close()                
            return resultados
        except Exception as e:
            print("Error en la consulta:", e)
            if conn:
                conn.close()

    def definirPrincipaProveSolimicro(self,artCompra):
        resultados = []        
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                for id in artCompra:
                    principal = 0
                    query = text(f"SELECT * FROM tbArticuloProveedor WHERE (IDArticulo = N'{id}')")
                    result = conn.execute(query).fetchall()
                    for row in result:
                        if row[4] == 1:
                            principal = 1
                            break
                    if principal == 0:
                        update_query = text(f"UPDATE TOP (1) tbArticuloProveedor SET Principal = {1} WHERE IDArticulo = N'{id}'")
                        conn.execute(update_query)
                conn.commit()
                conn.close()
            return resultados
        except Exception as e:
            print("Error en la consulta:", e)
            if conn:
                conn.close()

    
    def updatePrincipal(self,artCompra):
        resultados = []        
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                for id in artCompra:
                    principal = 0
                    query = text(f"UPDATE tbArticuloProveedor SET Principal = {0} WHERE IDArticulo = N'{id}'")
                    conn.execute(query)
                    conn.commit()
                    # for row in result:
                    #     if row[4] == 1:
                    #         principal = 1
                    #         break
                    # if principal == 0:
                    update_query = text(f"UPDATE TOP (1) tbArticuloProveedor SET Principal = {1} WHERE IDArticulo = N'{id}'")
                    conn.execute(update_query)
                    conn.commit()
                # conn.commit()
                conn.close()
            return resultados
        except Exception as e:
            print("Error en la consulta:", e)
            if conn:
                conn.close()

    def insertToTemp(self, datos):
        conn = None
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                for id in datos:
                    stringInsert = text(f"INSERT INTO a_quitar (IDArticulo) VALUES (N'{id}')")
                    conn.execute(stringInsert)
                    conn.commit()
        except Exception as e:
            print("Error de insercion", e)
            return None
        finally:
            if conn:
                conn.close()
    @staticmethod
    def export_to_excel(data):
        df = pd.DataFrame(data,columns=["IDArticulo"])
        df.to_excel("ArtiSinProveedor.xlsx",index=False)
    @staticmethod
    def export_to_excel_art_desd_indus(data):
        df = pd.DataFrame(data,columns=["CodigoArticulo","ProveedorHabitual","PrecioCosteStandard","PlazoAprovisionam"])
        df.to_excel("ArtiConProveedorIndustry.xlsx",index=False)
    @staticmethod
    def export_to_excel_art_sin_prove(data):
        df = pd.DataFrame(data,columns=["CodigoArticulo","ProveedorHabitual"])
        df.to_excel("ArtiSinProveedorSolmicro.xlsx",index=False)

In [150]:
iArtCompra = ArticuloProveedor()

In [129]:
# Otener todos los articulos de compra IDTipo = 04 IDFamilia >4000 and <4200 and <>4020
list_art_compra = iArtCompra.getArtCompra()
print(len(list_art_compra))
# iArtCompra.export_to_excel_art_desd_indus(data=arti_con_provee_desde_industry)
# iArtCompra.insertToTemp(datos=list_art_sin_proveedor)
# iArtCompra.export_to_excel(list_art_sin_proveedor)

6585
7651


In [139]:
# Obtener todos los articulos de compra IDTipo = 04 IDFamilia >4000 and <4200 and <>4020 de tabla tbArticuloProveedor
list_art_con_proveedor = iArtCompra.getArtCompraConProveedor()
print(len(list_art_con_proveedor))

7651


In [151]:
iArtCompra.updatePrincipal(list_art_con_proveedor)

[]

In [131]:
# Obtener los articulos sin proveedor
list_art_sin_proveedor = list(set(list_art_compra)-set(list_art_con_proveedor))
print(len(list_art_sin_proveedor))

497


In [133]:
arti_con_provee_desde_industry = iArtCompra.getProArtConProveeDesdeIndustry(list_ids=list_art_sin_proveedor)

In [107]:
checkProveedor = iArtCompra.getCheckProveedor(listaProveedor=proveedores)

In [120]:
checkProveedor

[31, 1090, 1092, 1093, 1096, 75, 614, 1126, 1133, 1139, 163]